In [13]:
import threading
import time
import psutil
import os

def evaluate_thread_sampler(sampler_class, test_function, iterations=10):
    """
    评估ThreadSampler类的性能开销
    
    参数:
        sampler_class: ThreadSampler类
        test_function: 要测试的目标函数
        iterations: 测试迭代次数
        interval: 采样间隔
        
    返回:
        包含各项指标的字典（中文输出）
    """
    # 基准测试（无采样）
    print("正在进行基准测试（无采样）...")
    base_times = []
    base_cpu_usages = []
    base_mem_usages = []
    
    for _ in range(iterations):
        # 获取当前进程
        process = psutil.Process(os.getpid())
        
        # 记录开始状态
        start_cpu = process.cpu_percent(interval=None)
        start_mem = process.memory_info().rss / 1024 / 1024  # MB
        start_time = time.time()
        
        # 执行测试函数
        test_function()
        
        # 记录结束状态
        end_time = time.time()
        end_cpu = process.cpu_percent(interval=None)
        end_mem = process.memory_info().rss / 1024 / 1024  # MB
        
        # 保存指标
        base_times.append(end_time - start_time)
        base_cpu_usages.append(end_cpu - start_cpu)
        base_mem_usages.append(end_mem - start_mem)
    
    # 采样测试
    print("正在进行采样测试...")
    sampled_times = []
    sampled_cpu_usages = []
    sampled_mem_usages = []
    
    for _ in range(iterations):
        # 初始化采样器
        sampler = sampler_class()
        
        # 获取当前进程
        process = psutil.Process(os.getpid())
        
        # 记录开始状态
        start_cpu = process.cpu_percent(interval=None)
        start_mem = process.memory_info().rss / 1024 / 1024  # MB
        start_time = time.time()
        
        # 启动采样并执行测试函数
        sampler.start()
        # sampler.enable()
        test_function()
        sampler.stop()
        # sampler.disable()
        
        # 记录结束状态
        end_time = time.time()
        end_cpu = process.cpu_percent(interval=None)
        end_mem = process.memory_info().rss / 1024 / 1024  # MB
        
        # 保存指标
        sampled_times.append(end_time - start_time)
        sampled_cpu_usages.append(end_cpu - start_cpu)
        sampled_mem_usages.append(end_mem - start_mem)
    
    # 计算指标
    avg_base_time = sum(base_times) / iterations
    avg_sampled_time = sum(sampled_times) / iterations
    time_increase_percent = ((avg_sampled_time - avg_base_time) / avg_base_time) * 100
    
    avg_base_cpu = sum(base_cpu_usages) / iterations
    avg_sampled_cpu = sum(sampled_cpu_usages) / iterations
    cpu_overhead = avg_sampled_cpu - avg_base_cpu
    
    avg_base_mem = sum(base_mem_usages) / iterations
    avg_sampled_mem = sum(sampled_mem_usages) / iterations
    mem_overhead = avg_sampled_mem - avg_base_mem
    
    # 输出结果（中文）
    print("\n性能评估结果:")
    print(f"1. 执行时间增加百分比: {time_increase_percent:.2f}%")
    print(f"   - 基准执行时间: {avg_base_time:.6f}秒")
    print(f"   - 采样执行时间: {avg_sampled_time:.6f}秒")
    
    print(f"2. CPU开销: {cpu_overhead:.2f}%")
    print(f"   - 基准CPU使用率: {avg_base_cpu:.2f}%")
    print(f"   - 采样CPU使用率: {avg_sampled_cpu:.2f}%")
    
    # print(f"3. 内存开销: {mem_overhead:.2f}MB")
    # print(f"   - 基准内存使用: {avg_base_mem:.2f}MB")
    # print(f"   - 采样内存使用: {avg_sampled_mem:.2f}MB")
    
    return {
        "执行时间增加百分比": time_increase_percent,
        "CPU开销": cpu_overhead,
        "内存开销": mem_overhead,
        "基准执行时间": avg_base_time,
        "采样执行时间": avg_sampled_time,
        "基准CPU使用率": avg_base_cpu,
        "采样CPU使用率": avg_sampled_cpu,
        "基准内存使用": avg_base_mem,
        "采样内存使用": avg_sampled_mem
    }

# 示例测试函数
def worker(num):
    for _ in range(3):
        # print(f"Thread {num}: Working...")
        time.sleep(0.1)

def test_function():
    for i in range(1000000):
        _ = i * i

# 使用示例
if __name__ == "__main__":
    # 评估ThreadSampler类
    from profiler import ThreadSampler
    from cProfile import Profile

    results = evaluate_thread_sampler(ThreadSampler, test_function, iterations=5)
    # results = evaluate_thread_sampler(Profile, test_function, iterations=5)
    
    # 可以保存结果到文件
    # import json
    # with open("sampler_evaluation.json", "w") as f:
    #     json.dump(results, f, indent=4)

正在进行基准测试（无采样）...
正在进行采样测试...

性能评估结果:
1. 执行时间增加百分比: 9.04%
   - 基准执行时间: 0.029384秒
   - 采样执行时间: 0.032041秒
2. CPU开销: 4.76%
   - 基准CPU使用率: 94.82%
   - 采样CPU使用率: 99.58%
